# 1.Base model. 
60 seconds x11 features, processed by CNN, parquet uploaded from binance 

In [1]:

import pandas as pd
import numpy as np
from numpy import array
import talib
import matplotlib
from datetime import datetime, timedelta
import time
import glob
import os
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, BatchNormalization,LeakyReLU
from tensorflow.keras import regularizers
from tensorflow import keras
from tensorflow.keras import callbacks
from tensorflow.keras import backend
import psutil
import requests
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn import model_selection, metrics

min_max_scaler = preprocessing.MinMaxScaler()
 
time_slot_front = 1 # minutes
time_slot_wave = 1 # minutes
query_condition = 'volume > 250'  # 400  100
query_condition2 = 'volume > 100'  # 100   25 BNB
expected_percentage = 0.5
 
path_imp = 'C:/data_raw/train/*.csv'
path_exp = 'C:/data_pro/'
x_train_name = "train_x.npy"
y_train_name = "train_y.npy"
x_test_name = "test_x.npy"
y_test_name = "test_y.npy"


train_list = ['2022-03','2022-04', '2022-05', '2022-06']    
test_list= ['2022-07']

train_list_files = []
test_list_files=[]
for m in train_list:
    train_list_files.append(f'C:/data_raw/ETHUSDT-aggTrades-{m}.csv.parquet')
for m in test_list:
    test_list_files.append(f'C:/data_raw/ETHUSDT-aggTrades-{m}.csv.parquet')


features = 11
events = 61
start_time = time.time()

tf.random.set_seed(1)


def telegram(message):
    y = requests.post(f'http://api.telegram.org/bot1434154647:AAGf-SNF84zR7jQ7sGShQtJX9Jq5UurTKJI/sendMessage?chat_id=-379051501&text={message}')

def conf_matrix(y,pred):
    ((tn, fp), (fn, tp)) = metrics.confusion_matrix(y, pred)
    ((tnr,fpr),(fnr,tpr))= metrics.confusion_matrix(y, pred, normalize='true')
    return pd.DataFrame([[f'TN = {tn} (TNR = {tnr:1.2%})', f'FP = {fp} (FPR = {fpr:1.2%})'], 
                         [f'FN = {fn} (FNR = {fnr:1.2%})', f'TP = {tp} (TPR = {tpr:1.2%})']],
                        index=['True 0', 'True 1'], columns=['Pred 0','Pred 1'])


def prepare_data(read_files,mem):
    counter = 0
    d3 = np.zeros(shape=(mem,events, features,1), dtype=float)  # float 64 #d3 = np.zeros(shape=(mem,events, features)).astype('float32')  # float 32
    d1 = []
    for f in read_files:
        print('loading to memory',f,datetime.now())
        df_ini= pd.read_parquet(f)
        
        df_ini['Timestamp']=df_ini['Timestamp']//1000
        df_01 = df_ini.loc[df_ini['is_maker'] == True].copy()
        df_02 = df_01.groupby(
            ['Timestamp'],as_index=False).agg(
            C1=('Timestamp', 'count'),
            S1=('volume', 'sum'))

        df_11 = df_ini.loc[df_ini['is_maker'] == False].copy()
        df_12 = df_11.groupby(
            ['Timestamp'],as_index=False).agg(
            C2=('Timestamp', 'count'),
            S2=('volume', 'sum'))
        df_12 = df_12.assign(Timestamp = pd.to_datetime(df_12['Timestamp'],unit='s')).set_index('Timestamp')
        df_02 = df_02.assign(Timestamp = pd.to_datetime(df_02['Timestamp'],unit='s')).set_index('Timestamp')
        df_ini = df_ini.assign(Timestamp = pd.to_datetime(df_ini['Timestamp'],unit='s')).set_index('Timestamp')

        df_ini.drop(['tradeid','first_tradeid','last_tradeid','is_maker'], axis=1, inplace=True)
        df_ohlc= df_ini.resample('1S')['price'].ohlc()    #1S or 1min
        df_vol= df_ini.resample('1S')['volume'].sum()
        df_vol2= df_ini.resample('1S')['volume'].ohlc()
        df_vol2 = df_vol2.rename({'open': 'openv', 'close': 'closev', 'high': 'highv', 'low': 'lowv'}, axis=1)  # new method
        df = pd.concat([df_ohlc,df_vol,df_vol2,df_12,df_02], axis=1)


        df.reset_index(drop=True, inplace=True)
        df=df.fillna(method='ffill')
    
        df['rsi'] = talib.RSI(df.close,timeperiod=100)
        df['atr'] = talib.ATR(df.high, df.low, df.close)
        df['macd'] = talib.MACD(df.close)[1]
        
        
        df['returns'] = df.close.pct_change()*1000                     

 
        df = df.iloc[1800:,]          # delete first seconds without rsi
        df_unchanged = df.copy()

        df_base= df_unchanged.query(query_condition)          # cycle through only where expected volume (unchanged - no normalizarion)
        for index, row in df_base.iterrows():

            df_front =  df_unchanged.loc[index - (time_slot_front*60 ) : index].copy() 
            df_wave = df_unchanged.loc[index : index + time_slot_wave*60  -2].copy()  

            df_prefront = df_unchanged.loc[index - (time_slot_front*1800 ) : index].copy()
            my_mean = df_prefront['close'].mean()
            df_front['to_mean'] = df_front['close']/my_mean*100
            my_largest = df_prefront['close'].nlargest(3).mean()
            df_front['to_largest'] = df_front['close']/my_largest*100
            my_smallest = df_prefront['close'].nsmallest(3).mean()
            df_front['to_smallest'] = df_front['close']/my_smallest*100
            
            
            if len(df_front.index) == (time_slot_front*60 + 1) and len (df_wave.index)== (time_slot_wave*60 + 1-2):    
                max_price = df_wave['close'].max()
                min_price = df_wave['close'].min()
                initial_price = df_wave.iloc[0]['close']
                element_result = 0
                if max_price >= initial_price * 1.005:
                    try:
                        deal_index_pos = df_wave[df_wave['close'].gt(initial_price*1.005)].index[0]
                    except:
                        deal_index_pos = 0
                    try:
                        deal_index_neg = df_wave[df_wave['close'].lt(initial_price*0.995)].index[0]
                    except:
                        deal_index_neg = 0
                    if deal_index_pos < deal_index_neg: element_result = 1
                    if deal_index_neg == 0 and deal_index_pos >0: element_result = 1
                    
                    
                df_front.drop(['open','high','low','openv','highv','lowv','closev','volume','close'], axis=1, inplace=True)
 
                #df_front.drop(['S1','rsi','macd','to_mean','S2','returns'], axis=1, inplace=True)

                element_numpy = df_front.to_numpy()
                d3[counter] = element_numpy.reshape(1, events, features,1)  # оптимально добавляем с увеличением размерности массива до 3d
                d1.append(element_result)   #просто добавляем элемент массива
                counter += 1
                if counter % 15000 ==0: print(f, counter," frames per minute =" , round(counter*60/((time.time() - start_time)), 1))
    d1 = np.array(d1)
    d3= d3[:counter]
    print('final d3', d3.shape[0],'positive',np.sum(d1))
    print(list(df_front))
    return d3,d1

def conf_matrix(y,pred):
    ((tn, fp), (fn, tp)) = metrics.confusion_matrix(y, pred)
    ((tnr,fpr),(fnr,tpr))= metrics.confusion_matrix(y, pred, normalize='true')
    return pd.DataFrame([[f'TN = {tn} (TNR = {tnr:1.2%})', f'FP = {fp} (FPR = {fpr:1.2%})'], 
                         [f'FN = {fn} (FNR = {fnr:1.2%})', f'TP = {tp} (TPR = {tpr:1.2%})']],
                        index=['True 0', 'True 1'], columns=['Pred 0','Pred 1'])



x,y = prepare_data(train_list_files,2000000)

np.save (x_train_name,x)
np.save (y_train_name,y)

_x,_y = prepare_data(test_list_files,900000)

np.save (x_test_name,_x)
np.save (y_test_name,_y)




tf.random.set_seed(1)
if 'model' in globals():del model


for period in range(2):

    backend.clear_session()
    model = Sequential()
    model.add(Conv2D(filters=64, kernel_size = (4), activation='relu',input_shape=[events, features,1]))
    #model.add(MaxPooling2D())
    model.add(Flatten())
    #model.add(Dense(2048, activation='relu',activity_regularizer= tf.keras.regularizers.L1(l1=0.01))) 
    model.add(Dense(2048, activation='relu')) 
    #model.add(Dropout(0.3))                             
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                   optimizer='rmsprop',
                   metrics=['AUC'])
    backend.set_value(model.optimizer.learning_rate,0.0001)
    callback_list=[
        callbacks.EarlyStopping(monitor="val_loss",patience=5,restore_best_weights=True),
        callbacks.ReduceLROnPlateau(factor=0.8,monitor="val_loss",patience=3)]
    history = model.fit(x = x, y = y,  epochs=20,   validation_split=0.3,   batch_size= 70,callbacks=callback_list)
    model.save(f'opt_exp_21_{str(period)}.h5')

    predictions = model.predict(_x)  
    
    pred_tuned_thresh = np.where(predictions >= 0.75, 1, 0) 
    print(conf_matrix(_y,pred_tuned_thresh))
    


loading to memory C:/data_raw/ETHUSDT-aggTrades-2022-03.csv.parquet 2022-12-18 08:50:34.579234
C:/data_raw/ETHUSDT-aggTrades-2022-03.csv.parquet 15000  frames per minute = 14989.2
C:/data_raw/ETHUSDT-aggTrades-2022-03.csv.parquet 30000  frames per minute = 15929.3
loading to memory C:/data_raw/ETHUSDT-aggTrades-2022-04.csv.parquet 2022-12-18 08:52:34.998542
C:/data_raw/ETHUSDT-aggTrades-2022-04.csv.parquet 45000  frames per minute = 15720.7
loading to memory C:/data_raw/ETHUSDT-aggTrades-2022-05.csv.parquet 2022-12-18 08:54:07.576345
C:/data_raw/ETHUSDT-aggTrades-2022-05.csv.parquet 60000  frames per minute = 15315.5
C:/data_raw/ETHUSDT-aggTrades-2022-05.csv.parquet 75000  frames per minute = 15779.5
C:/data_raw/ETHUSDT-aggTrades-2022-05.csv.parquet 90000  frames per minute = 16098.4
C:/data_raw/ETHUSDT-aggTrades-2022-05.csv.parquet 105000  frames per minute = 16371.1
loading to memory C:/data_raw/ETHUSDT-aggTrades-2022-06.csv.parquet 2022-12-18 08:57:12.465024
C:/data_raw/ETHUSDT-aggT

In [ ]:
# we can continue with uploaded arrays and change hyperparamenters

x = np.load (x_train_name)
y = np.load (y_train_name)


_x = np.load (x_test_name)
_y = np.load (y_test_name)


tf.random.set_seed(1)
if 'model' in globals():del model


for period in range(2):

    backend.clear_session()
    model = Sequential()
    model.add(Conv2D(filters=64, kernel_size = (4), activation='relu',input_shape=[events, features,1]))
    #model.add(MaxPooling2D())
    model.add(Flatten())
    #model.add(Dense(2048, activation='relu',activity_regularizer= tf.keras.regularizers.L1(l1=0.01))) 
    model.add(Dense(2048, activation='relu')) 
    #model.add(Dropout(0.3))                             
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                   optimizer='rmsprop',
                   metrics=['AUC'])
    backend.set_value(model.optimizer.learning_rate,0.0001)
    callback_list=[
        callbacks.EarlyStopping(monitor="val_loss",patience=5,restore_best_weights=True),
        callbacks.ReduceLROnPlateau(factor=0.8,monitor="val_loss",patience=3)]
    history = model.fit(x = x, y = y,  epochs=20,   validation_split=0.3,   batch_size= 70,callbacks=callback_list)
    model.save(f'opt_exp_21_{str(period)}.h5')

    predictions = model.predict(_x)  
    
    pred_tuned_thresh = np.where(predictions >= 0.75, 1, 0) 
    print(conf_matrix(_y,pred_tuned_thresh))



In [ ]:
# prepare all coins CLOSE by seconds - we need FACTOR12

import requests
import zipfile, io
import pandas as pd
import sys
import os
import glob

periods = [ '2022-01', '2022-02', '2022-03', '2022-04','2022-05', '2022-06','2022-07','2022-08', '2022-09', '2022-10']  
coins = ['BCH','BNB','BTC','EOS','ETC','ETH','LTC','XMR','TRX','XLM','ADA','IOTA','MKR','DOGE']

#1

for period in periods:
    for coin in coins:
        coin_usd = coin + 'USDT'
        zip_file_url = f'https://data.binance.vision/data/futures/um/monthly/aggTrades/{coin_usd}/{coin_usd}-aggTrades-{period}.zip'
        print('Uploading & unpacking ',zip_file_url)
        try:
            r = requests.get(zip_file_url)
            z = zipfile.ZipFile(io.BytesIO(r.content))
            z.extractall("E:/feat12")
        except:
            print('no data')
 



read_files = glob.glob(f"E:/feat12/*.csv")
for f in read_files:
    df= pd.read_csv(f,sep=',',header=None,  skiprows=1,nrows=999999999,
                    names=['tradeid','price','volume','first_tradeid','last_tradeid','Timestamp','is_maker'])
    df['Timestamp']=df['Timestamp']//1000
    df.drop(['tradeid','first_tradeid','last_tradeid','is_maker'], axis=1, inplace=True)
    df = df.assign(Timestamp = pd.to_datetime(df['Timestamp'],unit='s')).set_index('Timestamp')
    df_sampled= df.resample('1S')['price'].ohlc()
    df_sampled=df_sampled.fillna(method='ffill')
    df_sampled.drop(['open','high','low'], axis=1, inplace=True)
    df_sampled.to_parquet(f+'.parquet') 
    print (f+'.parquet')

    
    



for coin in coins:
    read_files = glob.glob(f"E:/feat12/{coin}*.parquet")
    first_run = 1
    df = pd.DataFrame()
    for f in read_files:
        curr_column = f.split('USDT-aggTrades-')[0].split('\\')[1]
        print (f)
        df_ini = pd.read_parquet(f)
        df_ini  = df_ini.rename(columns={'close': curr_column})
        if first_run ==1:
            first_run = 0
            df = df_ini
            print ('A 0 - just copy')
        else:    
            print ('A 1 - axis=0')        
            df = pd.concat([df,df_ini], axis=0)
    
    df.to_parquet(f'E:/feat12/coins/{coin}.parquet')

    
df = pd.DataFrame()    
read_files = glob.glob(f"E:/feat12/coins/*.parquet") 
first_run = 1
for f in read_files:
    df_ini = pd.read_parquet(f)
    if len(df_ini) > 0 :
        if first_run ==1:
            df = df_ini
            first_run = 0
            print ('B 0 - just copy')
        else:
            print ('B 2 - axis=1')        
            df = pd.concat([df,df_ini], axis=1)
        
df.to_parquet('E:/feat12/full/full.parquet')        


# 2. Base model + FACTOR12

In [1]:

import pandas as pd
import numpy as np
from numpy import array
import talib
import matplotlib
from datetime import datetime, timedelta
import time
import glob
import os
import pickle
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, BatchNormalization,LeakyReLU
from tensorflow.keras import regularizers
from tensorflow import keras
from tensorflow.keras import callbacks
from tensorflow.keras import backend
import psutil
import requests
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn import model_selection, metrics

min_max_scaler = preprocessing.MinMaxScaler()
 
time_slot_front = 1 # minutes
time_slot_wave = 1 # minutes
query_condition = 'volume > 250'  # 400  100
query_condition2 = 'volume > 100'  # 100   25 BNB
expected_percentage = 0.5
 
path_imp = 'C:/data_raw/train/*.csv'
path_exp = 'C:/data_pro/'
x_train_name = "train_x.npy"
y_train_name = "train_y.npy"
x_test_name = "test_x.npy"
y_test_name = "test_y.npy"


train_list = ['2022-03','2022-04', '2022-05', '2022-06']    
test_list= ['2022-07']

train_list_files = []
test_list_files=[]
for m in train_list:
    train_list_files.append(f'C:/data_raw/ETHUSDT-aggTrades-{m}.csv.parquet')
for m in test_list:
    test_list_files.append(f'C:/data_raw/ETHUSDT-aggTrades-{m}.csv.parquet')


features = 20
events = 61
start_time = time.time()

# insertion 1
full_close = pd.read_parquet ('C:/data_raw/full/full.parquet')
full_close=full_close.fillna(method='ffill')
full_close = full_close.dropna()
#


tf.random.set_seed(1)


def telegram(message):
    y = requests.post(f'http://api.telegram.org/bot1434154647:AAGf-SNF84zR7jQ7sGShQtJX9Jq5UurTKJI/sendMessage?chat_id=-379051501&text={message}')

def conf_matrix(y,pred):
    ((tn, fp), (fn, tp)) = metrics.confusion_matrix(y, pred)
    ((tnr,fpr),(fnr,tpr))= metrics.confusion_matrix(y, pred, normalize='true')
    return pd.DataFrame([[f'TN = {tn} (TNR = {tnr:1.2%})', f'FP = {fp} (FPR = {fpr:1.2%})'], 
                         [f'FN = {fn} (FNR = {fnr:1.2%})', f'TP = {tp} (TPR = {tpr:1.2%})']],
                        index=['True 0', 'True 1'], columns=['Pred 0','Pred 1'])


def prepare_data(read_files,mem):
    counter = 0
    d3 = np.zeros(shape=(mem,events, features,1), dtype=float)  # float 64 #d3 = np.zeros(shape=(mem,events, features)).astype('float32')  # float 32
    d1 = []
    for f in read_files:
        print('loading to memory',f,datetime.now())
        df_ini= pd.read_parquet(f)
        
        df_ini['Timestamp']=df_ini['Timestamp']//1000
        df_01 = df_ini.loc[df_ini['is_maker'] == True].copy()
        df_02 = df_01.groupby(
            ['Timestamp'],as_index=False).agg(
            C1=('Timestamp', 'count'),
            S1=('volume', 'sum'))

        df_11 = df_ini.loc[df_ini['is_maker'] == False].copy()
        df_12 = df_11.groupby(
            ['Timestamp'],as_index=False).agg(
            C2=('Timestamp', 'count'),
            S2=('volume', 'sum'))
        df_12 = df_12.assign(Timestamp = pd.to_datetime(df_12['Timestamp'],unit='s')).set_index('Timestamp')
        df_02 = df_02.assign(Timestamp = pd.to_datetime(df_02['Timestamp'],unit='s')).set_index('Timestamp')
        df_ini = df_ini.assign(Timestamp = pd.to_datetime(df_ini['Timestamp'],unit='s')).set_index('Timestamp')

        df_ini.drop(['tradeid','first_tradeid','last_tradeid','is_maker'], axis=1, inplace=True)
        df_ohlc= df_ini.resample('1S')['price'].ohlc()    #1S or 1min
        df_vol= df_ini.resample('1S')['volume'].sum()
        df_vol2= df_ini.resample('1S')['volume'].ohlc()
        df_vol2 = df_vol2.rename({'open': 'openv', 'close': 'closev', 'high': 'highv', 'low': 'lowv'}, axis=1)  # new method
        df = pd.concat([df_ohlc,df_vol,df_vol2,df_12,df_02], axis=1)

        # insertion 2  (factor 12)
        df =  pd.concat ([df,full_close],axis=1 ,join='inner' )
        df['f012'] = df['close']/(df['BCH'] + df['BNB'] + df['BTC'] + df['EOS'] + df['ETC']  + df['ETH'] + 
                                  df['LTC'] + df['XMR'] + df['TRX'] + df['XLM'] + df['ADA'] + df['IOTA'] + df['MKR']  + df['DOGE'])  
        lags = [60,300,900,1800]
        for lag in lags:
            df[f'f012_max_{lag}'] = df['f012'].rolling(lag).max()
            df[f'f012_min_{lag}'] = df['f012'].rolling(lag).min()

        df.drop(['BCH','BNB','BTC','EOS','ETC','ETH','LTC','XMR','TRX','XLM','ADA','IOTA','MKR','DOGE'], axis=1, inplace=True)
        #

        df.reset_index(drop=True, inplace=True)
        df=df.fillna(method='ffill')
    
        df['rsi'] = talib.RSI(df.close,timeperiod=100)
        df['atr'] = talib.ATR(df.high, df.low, df.close)
        df['macd'] = talib.MACD(df.close)[1]
        
        
        df['returns'] = df.close.pct_change()*1000                     

 
        df = df.iloc[1800:,]          # delete first seconds without rsi
        df_unchanged = df.copy()

        df_base= df_unchanged.query(query_condition)          # cycle through only where expected volume (unchanged - no normalizarion)
        for index, row in df_base.iterrows():

            df_front =  df_unchanged.loc[index - (time_slot_front*60 ) : index].copy() 
            df_wave = df_unchanged.loc[index : index + time_slot_wave*60  -2].copy()  

            df_prefront = df_unchanged.loc[index - (time_slot_front*1800 ) : index].copy()
            my_mean = df_prefront['close'].mean()
            df_front['to_mean'] = df_front['close']/my_mean*100
            my_largest = df_prefront['close'].nlargest(3).mean()
            df_front['to_largest'] = df_front['close']/my_largest*100
            my_smallest = df_prefront['close'].nsmallest(3).mean()
            df_front['to_smallest'] = df_front['close']/my_smallest*100
            
            
            if len(df_front.index) == (time_slot_front*60 + 1) and len (df_wave.index)== (time_slot_wave*60 + 1-2):    
                max_price = df_wave['close'].max()
                min_price = df_wave['close'].min()
                initial_price = df_wave.iloc[0]['close']
                element_result = 0
                if max_price >= initial_price * 1.005:
                    try:
                        deal_index_pos = df_wave[df_wave['close'].gt(initial_price*1.005)].index[0]
                    except:
                        deal_index_pos = 0
                    try:
                        deal_index_neg = df_wave[df_wave['close'].lt(initial_price*0.995)].index[0]
                    except:
                        deal_index_neg = 0
                    if deal_index_pos < deal_index_neg: element_result = 1
                    if deal_index_neg == 0 and deal_index_pos >0: element_result = 1
                    
                    
                df_front.drop(['open','high','low','openv','highv','lowv','closev','volume','close'], axis=1, inplace=True)
 
                #df_front.drop(['S1','rsi','macd','to_mean','S2','returns'], axis=1, inplace=True)

                element_numpy = df_front.to_numpy()
                d3[counter] = element_numpy.reshape(1, events, features,1)  # оптимально добавляем с увеличением размерности массива до 3d
                d1.append(element_result)   #просто добавляем элемент массива
                counter += 1
                if counter % 15000 ==0: print(f, counter," frames per minute =" , round(counter*60/((time.time() - start_time)), 1))
    d1 = np.array(d1)
    d3= d3[:counter]
    print('final d3', d3.shape[0],'positive',np.sum(d1))
    print(list(df_front))
    return d3,d1

def conf_matrix(y,pred):
    ((tn, fp), (fn, tp)) = metrics.confusion_matrix(y, pred)
    ((tnr,fpr),(fnr,tpr))= metrics.confusion_matrix(y, pred, normalize='true')
    return pd.DataFrame([[f'TN = {tn} (TNR = {tnr:1.2%})', f'FP = {fp} (FPR = {fpr:1.2%})'], 
                         [f'FN = {fn} (FNR = {fnr:1.2%})', f'TP = {tp} (TPR = {tpr:1.2%})']],
                        index=['True 0', 'True 1'], columns=['Pred 0','Pred 1'])

x,y = prepare_data(train_list_files,2000000)
np.save (x_train_name,x)
np.save (y_train_name,y)

_x,_y = prepare_data(test_list_files,900000)
np.save (x_test_name,_x)
np.save (y_test_name,_y)


tf.random.set_seed(1)
if 'model' in globals():del model

for period in range(2):
    backend.clear_session()
    model = Sequential()
    model.add(Conv2D(filters=64, kernel_size = (4), activation='relu',input_shape=[events, features,1]))
    model.add(Flatten())
    model.add(Dense(2048, activation='relu')) 
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                   optimizer='rmsprop',
                   metrics=['AUC'])
    backend.set_value(model.optimizer.learning_rate,0.0001)
    callback_list=[
        callbacks.EarlyStopping(monitor="val_loss",patience=5,restore_best_weights=True),
        callbacks.ReduceLROnPlateau(factor=0.8,monitor="val_loss",patience=3)]
    history = model.fit(x = x, y = y,  epochs=20,   validation_split=0.3,   batch_size= 70,callbacks=callback_list)
    model.save(f'opt_exp_21_{str(period)}.h5')

    predictions = model.predict(_x)  
    
    pred_tuned_thresh = np.where(predictions >= 0.75, 1, 0) 
    print(conf_matrix(_y,pred_tuned_thresh))
   


loading to memory C:/data_raw/ETHUSDT-aggTrades-2022-03.csv.parquet 2022-12-20 16:21:16.055503
C:/data_raw/ETHUSDT-aggTrades-2022-03.csv.parquet 15000  frames per minute = 11943.3
C:/data_raw/ETHUSDT-aggTrades-2022-03.csv.parquet 30000  frames per minute = 14161.4
loading to memory C:/data_raw/ETHUSDT-aggTrades-2022-04.csv.parquet 2022-12-20 16:23:24.578199
C:/data_raw/ETHUSDT-aggTrades-2022-04.csv.parquet 45000  frames per minute = 14253.1
loading to memory C:/data_raw/ETHUSDT-aggTrades-2022-05.csv.parquet 2022-12-20 16:25:00.899004
C:/data_raw/ETHUSDT-aggTrades-2022-05.csv.parquet 60000  frames per minute = 14052.5
C:/data_raw/ETHUSDT-aggTrades-2022-05.csv.parquet 75000  frames per minute = 14633.6
C:/data_raw/ETHUSDT-aggTrades-2022-05.csv.parquet 90000  frames per minute = 15047.9
C:/data_raw/ETHUSDT-aggTrades-2022-05.csv.parquet 105000  frames per minute = 15343.2
loading to memory C:/data_raw/ETHUSDT-aggTrades-2022-06.csv.parquet 2022-12-20 16:28:14.930667
C:/data_raw/ETHUSDT-aggT


KeyboardInterrupt



In [2]:
# we can continue with uploaded arrays and change hyperparamenters

x = np.load (x_train_name)
y = np.load (y_train_name)


_x = np.load (x_test_name)
_y = np.load (y_test_name)





for period in range(2):
    '''
    backend.clear_session()
    model = Sequential()
    model.add(Conv2D(filters=64, kernel_size = (4), activation='relu',input_shape=[events, features,1]))
    #model.add(MaxPooling2D())
    model.add(Flatten())
    #model.add(Dense(2048, activation='relu',activity_regularizer= tf.keras.regularizers.L1(l1=0.01))) 
    model.add(Dense(2048, activation='relu')) 
    #model.add(Dropout(0.3))                             
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy',
                   optimizer='rmsprop',
                   metrics=['AUC'])
    backend.set_value(model.optimizer.learning_rate,0.0001)
    callback_list=[
        callbacks.EarlyStopping(monitor="val_loss",patience=5,restore_best_weights=True),
        callbacks.ReduceLROnPlateau(factor=0.8,monitor="val_loss",patience=3)]
    history = model.fit(x = x, y = y,  epochs=20,   validation_split=0.3,   batch_size= 70,callbacks=callback_list)
    model.save(f'opt_exp_21_{str(period)}.h5')
    '''

    predictions = model.predict(_x)  
    
    pred_tuned_thresh = np.where(predictions >= 0.75, 1, 0) 
    print(conf_matrix(_y,pred_tuned_thresh))


5531/5531 [==============================] - 34s 6ms/step
                             Pred 0                Pred 1
True 0  TN = 169428 (TNR = 100.00%)  FP = 0 (FPR = 0.00%)
True 1    FN = 7543 (FNR = 100.00%)  TP = 0 (TPR = 0.00%)
5531/5531 [==============================] - 34s 6ms/step
                             Pred 0                Pred 1
True 0  TN = 169428 (TNR = 100.00%)  FP = 0 (FPR = 0.00%)
True 1    FN = 7543 (FNR = 100.00%)  TP = 0 (TPR = 0.00%)
